In [ ]:

#from Segementation import segment_face
#from ESLR_Net import ESLR_Net
#from model_bel import model_bel
#from VGGNet import VGGNet
#from AlexNet import AlexNet
#from GoogLeNet import GoogLeNet
import os 
import easydict
import matplotlib.pyplot as plt
from imutils import paths
import cv2
import numpy as np
import argparse
from keras.losses import categorical_crossentropy
from keras.preprocessing.image import img_to_array
from keras.optimizers import Adam, Adamax
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from PIL import Image


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
batch_size = 32
num_epoch = 50
image_size = 128
IMAGE_DIMS = (128, 128, 3)#if u use normal RGB then the dimension will be 3 or if u applay segementation the dimension will be 1

In [ ]:
print("[INFO] loading images...")
imagePaths = paths.list_images('/content/drive/My Drive/Implementation detection/Dataset')
data = []
labels = []
# loop over our input images
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    # load the input image from disk, resize it to 200x200 pixels, scale
    image = cv2.imread(imagePath)   # if you run with out segmentation 
    #image = cv2.imdecode(np.fromfile(imagePath, dtype=np.uint8), 0) # if you applly segmentation
    #image = cv2.resize(image, (128,128))
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    # image = Segment_face.face_Segment(image)
    #image = Segment_skin.skin_Segment(image)
    #image = Segmentation.segment_steem(image)
    #filters = gabfilter.build_filters()
    #image = gabfilter.process(image, filters)
    #print(imagePath)
    #plt.imshow(image)

    image = img_to_array(image)        
    
    data.append(image)
     
    # extract the class label from the file path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)


[INFO] loading images...


In [ ]:
# encode the labels, converting them from strings to integers
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
 
# perform a training and testing split, using 70% of the data for
# training and 30% for evaluation
(trainX, testX, trainY, testY) = train_test_split(np.array(data), np.array(labels), test_size=0.30)
#(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25)

In [ ]:
print(testY.shape)
print(trainX.shape)
#plt.imshow(trainX[1])

(927, 12)
(2163, 128, 128, 3)


In [ ]:
# initialize the model
print("[INFO] compiling model...")
#  ESLRNet, SRNet_model, ESLR_model,  AlexNet,   VGGNet, GoogLeNet,  ResNet, ESLR1, ESLR2, ESLR3
model =SRNet_model.build_model(width=128, height=128, depth=IMAGE_DIMS[2], classes=len(lb.classes_))
#model =ESLR_Net.build_model()
model.summary()
# train the model using the Adam optimizer
print("[INFO] training network...")
opt = Adam(lr=1e-3, decay=1e-3 / 50)
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])

**Data** **Augumentation**

In [ ]:
aug = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, 
                         fill_mode="nearest")

aug1 = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")
aug2 = ImageDataGenerator(
	rotation_range=30, 
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY,
                                 batch_size=batch_size),
                        validation_data=(testX, testY),
                        steps_per_epoch=len(trainX),
                        epochs=50, verbose=1)


In [ ]:
print("[INFO] training network...")
H = model.fit(trainX,trainY,
          batch_size=batch_size,
          epochs=50,
          validation_data=(testX, testY),
          shuffle=True)

In [ ]:
print("[INFO] evaluating network...")
#Classes= ['class 1', 'class 2', 'class 3', 'class 4', 'class 5', 'class 6', 'class 7', 'class 8', 'class 9', 'class 10', 'class 11']
predictions = model.predict(testX, batch_size=batch_size)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

scores = model.evaluate(testX, testY, batch_size=batch_size, verbose=1)
print('\nTest Result: %.3f loss: %.3f' % (scores[1]*100, scores[0]))

In [ ]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = 50
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")

In [ ]:
# summarize history for accuracy
plt.plot(H.history['accuracy'])
plt.plot(H.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(H.history['loss'])
plt.plot(H.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save('/content/drive/My Drive/Implementation steemrust/Saved_models/Steem_model.h5')
print('model saved to disk')

model saved to disk


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from keras.models import load_model
model = load_model('/content/drive/My Drive/Implementation steemrust/Saved_models/Steem_model.h5')
#image1 = '/content/drive/My Drive/Implementation steemrust/Dataset/100S/image (1).PNG'
#img_array =cv2.imread(image1)
#plt.imshow(img_array)
#plt.show()

In [ ]:
imagePaths = paths.list_images('/content/drive/My Drive/Implementation steemrust/Dataset')
data = []
labels = []
# loop over our input images
for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)  
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

image_testing = cv2.imread('/content/drive/My Drive/Implementation steemrust/Dataset/40MR/image14.png')
image_testing = cv2.resize(image_testing, (224, 224))
#image_testing = img_to_array(image_testing)        
image_testing.shape
plt.imshow(image_testing)
plt.show()
image_testing = np.expand_dims(image_testing,axis=0)
print(image_testing.shape)
output = model.predict(image_testing)
print(output)
print(lb.classes_[output.argmax(axis=1)])

In [ ]:
model_json=model.to_json()
with open("Models/model.json",'w') as json_file:
    json_file.write(model_json)
    
model.save_weights("Models/model_wal.h5")
print("saved model to disk")


In [ ]:
from keras.models import model_from_json
json_file = open('Models/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("Models/model_wal.h5")
print("Loaded model from disk")

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('Testing/awo.jpg', target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result =loaded_model.predict(test_image)

In [ ]:
a = [[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
b = [[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
c = [[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
d = [[0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
e = [[0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
f = [[0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
g = [[0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
h = [[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
i = [[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
k = [[0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
l = [[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
m = [[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
n = [[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
o = [[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
p = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
q = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
r = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]]
s = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]]
t = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]]
u = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]]
v = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]]
w = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]]
x = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]]
y = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]]
A = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]]
B = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]]
C = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]]
D = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0]]
E = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]]
F = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]]


In [ ]:
result=result.astype (int)
print (result)
result = result.tolist()
print (result)

In [ ]:
#training_set.class_indices

if(result==a):
    label=1
    print ("the gesture predicted is አዎ")

elif(result==b):
    label=2
    print ("the gesture predicted is አይ ")

elif(result==c):
    label=3
    print ("the gesture predicted is ትምህርት ቤት ")

elif (result==d):
    label=4
    print ("the gesture predicted is እንጀራ ")
elif (result==e):
    label=5
    print ("the gesture predicted is ጣፋጭ ")
elif (result==f):
    label=6
    print ("the gesture predicted is መራራ ")
elif (result==g):
    label=7
    print ("the gesture predicted is መዝናኛ ")
elif (result==h):
    label=8
    print ("the gesture predicted is አውነት ")
elif (result==i):
    label=9
    print ("the gesture predicted is ማመን ")
elif (result==k):
    label=10
    print ("the gesture predicted is K ")
elif (result==l):
    label=11
    print ("the gesture predicted is L ")
elif (result==m):
    label=12
    print ("the gesture predicted is M ")
elif (result==n):
    label=13
    print ("the gesture predicted is N ")
elif (result==o):
    label=14
    print ("the gesture predicted is O ")
elif (result==p):
    label=15
    print ("the gesture predicted is P ")
elif (result==q):
    label=16
    print ("the gesture predicted is Q ")
elif (result==r):
    label=17
    print ("the gesture predicted is R ")
elif (result==s):
    label=18
    print ("the gesture predicted is S ")
elif (result==t):
    label=19
    print ("the gesture predicted is T ")
elif (result==u):
    label=20
    print ("the gesture predicted is U ")
elif (result==v):
    label=21
    print ("the gesture predicted is V ")
elif (result==w):
    label=22
    print ("the gesture predicted is W ")
elif (result==x):
    label=23
    print ("the gesture predicted is X ")
elif (result==y):
    label=24
    print ("the gesture predicted is Y ")
    
else:
    print("No image predicted")
    
print ("Label of the gesture is",label)


In [ ]:
# extract and plot each detected face in a photograph
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from mtcnn.mtcnn import MTCNN
import cv2
y1=0
x1=0
width=0
height=0
x2=0
y2=0
class Segment_face():
    def face_Segment(img):
        detector = MTCNN()
        # detect faces in the image
        faces = detector.detect_faces(img)
        #print(len(faces))
        for i in range(len(faces)):
            # get coordinates
            x1, y1, width, height = faces[i]['box']
            x2, y2 = x1 + width, y1 + height
        img=img[y1:y2, x1:x2]
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        print('success')
        return img



Face Segmentation

In [ ]:
import cv2
from cv2 import destroyAllWindows
from cv2 import CascadeClassifier
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
class Segment_face():
    def face_Segment(img):

      detector = CascadeClassifier('/content/drive/My Drive/Dataset/haarcascade_frontalface_default.xml')
      # perform face detection
      faces = detector.detectMultiScale(img)
      for i in range(len(faces)):
        # get coordinates
        x1, y1, width, height = faces[i]
        x2, y2 = x1 + width, y1 + height
      data=img[y1:y2, x1:x2]
      gray = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
      print('success')
      return img

In [ ]:
print("[INFO] loading images...")

f_imagePaths = paths.list_images('/content/drive/My Drive/Dataset/Facial_Dataset')
f_data = []
f_labels = []

# loop over our input images
for f_imagePath in f_imagePaths:
    # load the image, pre-process it, and store it in the data list
    # load the input image from disk, resize it to 200x200 pixels, scale
    # the pixel intensities to the range [0, 1], and then update our
    # images list
    f_image = cv2.imread(f_imagePath)
    #image = cv2.resize(image, (224,224))
    f_image = cv2.resize(f_image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    #f_image = Segment_face.face_Segment(f_image)
    #print(f_imagePath)
    #plt.imshow(image)
    #f_image=cv2.resize(image,(48,48)
    f_image = img_to_array(f_image)        
    f_data.append(f_image)
     
    # extract the class label from the file path and update the
    # labels list
    f_label = f_imagePath.split(os.path.sep)[-2]
    f_labels.append(f_label)
    
f_data = np.array(f_data, dtype="float") / 255.0
f_labels = np.array(f_labels)
    
#plt.imshow(image)

#cv2.imshow('Image.jpg',image)
#cv2.waitKey()
#cv2.destroyAllWindows(0)



In [ ]:
# encode the labels, converting them from strings to integers
f_lb = LabelBinarizer()
f_labels = f_lb.fit_transform(f_labels)

# perform a training and testing split, using 75% of the data for
# training and 25% for evaluation
(f_trainX, f_testX, f_trainY, f_testY) = train_test_split(np.array(f_data),
	np.array(f_labels), test_size=0.25)
#(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25)

In [ ]:
print(f_trainX.shape)
print(f_trainY.shape)
plt.imshow(f_trainX[1])

In [ ]:
# initialize the model
print("[INFO] compiling model...")
f_model = model_bel.build_model(width=128, height=128,
    depth=IMAGE_DIMS[2], classes=len(f_lb.classes_))
f_model.summary()

In [ ]:
# train the model using the Adam optimizer
print("[INFO] training network...")
f_opt = Adam(lr=1e-3, decay=1e-3 / 50)
f_model.compile(loss="categorical_crossentropy", optimizer=f_opt,
    metrics=["accuracy"])

In [ ]:
f_H = f_model.fit(f_trainX,f_trainY,
          batch_size=batch_size,
          epochs=num_epoch,
          validation_data=(f_testX, f_testY),
          shuffle=True)

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
f_predictions = f_model.predict(f_testX, batch_size=batch_size)
print(classification_report(f_testY.argmax(axis=1),
	f_predictions.argmax(axis=1), target_names=f_lb.classes_))

f_scores = f_model.evaluate(f_testX, f_testY, batch_size=batch_size, verbose=1)
print('\nTest Result: %.3f loss: %.3f' % (f_scores[1]*100, f_scores[0]))

In [ ]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = num_epoch
plt.plot(np.arange(0, N), f_H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), f_H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), f_H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), f_H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")

In [ ]:
# ResNet trained on image size ?(224, 224)
# import the necessary packages
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import add
from keras.regularizers import l2
from keras import backend as K

class ResNet:
	@staticmethod
	def resid_module(data, K, stride, chanDim, red=False, reg=0.0001, bnEps=2e-5, bnMom=0.9):
		# the shortcut branch of the ResNet module should be 
		# initialized as the input (identity) data
		shortcut = data

		# the first block of the ResNet module are the 1x1 CONVs
		bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(data)
		act1 = Activation("relu")(bn1)
		conv1 = Conv2D(int(K*0.25), (1,1), use_bias=False, kernel_regularizer=l2(reg))(act1)

		# the second block of the ResNet module are the 3x3 CONVs
		bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv1)
		act2 = Activation("relu")(bn2)
		conv2 = Conv2D(int(K*0.25), (3,3), strides=stride, padding="same", use_bias=False, kernel_regularizer=l2(reg))(act2)

		# the third block of the ResNet module(another set of 1x1 CONVs)
		bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv2)
		act3 = Activation("relu")(bn3)
		conv3 = Conv2D(K, (1,1), use_bias=False, kernel_regularizer=l2(reg))(act3)

		# if we are to reduce the spatial size,
		# apply a CONV layer to the shortcut
		if red:
			shortcut = Conv2D(K, (1,1), strides=stride, use_bias=False, kernel_regularizer=l2(reg))(act1)

		# add shortcut and the final CONV
		x = add([conv3, shortcut])

		# return the addition as the output of the ResNet module
		return x

	@staticmethod
	def build_model(width, height, depth, classes, stages=(3,4,6), filters=(64,128,256,512), reg=0.0001, bnEps=2e-5, bnMom=0.9):
		# initialize the input shape to be "channels last" and 
		# the channels dimension itself
		inputShape = (height, width, depth)
		chanDim = -1

		# if channels first, update the input shape and channel dim
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# set the input and apply BN
		inputs = Input(shape=inputShape)
		x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(inputs)

		''' for cifar-10 dataset
		x = Conv2D(filters[0], (3, 3), use_bias=False, padding="same", kernel_regularizer=l2(reg))(x)
		'''

		# for Image Dataset
		# CONV => BN => ACT => POOL 
		x = Conv2D(filters[0], (5, 5), use_bias=False, padding="same", kernel_regularizer=l2(reg))(x)
		x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
		x = Activation("relu")(x)
		x = ZeroPadding2D((1, 1))(x)
		x = MaxPooling2D((3, 3), strides=(2,2))(x)

		# loop over the number of stages
		for i in range(0, len(stages)):
			# initialize the stride, then apply a residual module
			# used to reduce the spatial size of the input volume
			stride = (1,1) if i == 0 else (2,2)
			x = ResNet.resid_module(x, filters[i+1], stride, chanDim, red=True, bnEps=bnEps, bnMom=bnMom)

			# loop over the number of layers in the stage
			for j in range(0, stages[i] - 1):
				# apply a ResNet module
				x = ResNet.resid_module(x, filters[i+1], (1,1), chanDim, bnEps=bnEps, bnMom=bnMom)

		# apply BN => ACT => POOL
		x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
		x = Activation("relu")(x)
		x = AveragePooling2D(5,5)(x)

		# softmax classifier
		x = Flatten()(x)
		x = Dense(classes, kernel_regularizer=l2(reg))(x)
		x = Activation("softmax")(x)

		# create the model
		model = Model(inputs, x)

		# return the constructed network architectures
		return model

Skinn segment

In [ ]:
import cv2
import numpy as np
class Segment_skin():
    def skin_Segment(img):

        img_YCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
        YCrCb_mask = cv2.inRange(img_YCrCb, (0, 135, 85), (255,180,135)) 
        YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
    
        YCrCb_result = cv2.bitwise_not(YCrCb_mask)
    
        img2=img
        for i in range(img2.shape[0]):
            for j in range(img2.shape[1]):
                if YCrCb_result[i,j] != 0:
                    img2[i,j,0] = 0
                    img2[i,j,1] = 0
                    img2[i,j,2] = 0
    
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
        YCrCb_mask = cv2.erode(YCrCb_mask, kernel, iterations = 2)
        YCrCb_mask = cv2.dilate(YCrCb_mask, kernel, iterations = 2)
        print('sucess')
        return img2
        #return YCrCb_result
print('success')

success


In [ ]:
# GoogLeNet - 16 trained on image size (224, 224)
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.layers import concatenate
from keras.regularizers import l2
from keras import backend as K

class GoogLeNet:
	@staticmethod
	def conv_module(x, K, kX, kY, stride, chanDim, padding="same", reg=0.0005):
		#initialize the CONV, BN, and RELU layer names

		# define a CONV => BN => RELU pattern
		x = Conv2D(K, (kX, kY), strides=stride, padding=padding, kernel_regularizer=l2(reg))(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = Activation("relu")(x)

		# return the block
		return x

	@staticmethod
	def inception_mod(x, num1x1, num3x3Reduce, num3x3, num5x5Reduce, num5x5, num1x1Proj, chanDim, reg=0.0005):
		# define the first branch of the Inception module consisting
		# of 1 x 1 convolutions
		first = GoogLeNet.conv_module(x, num1x1, 1, 1, (1,1), chanDim, reg=reg)
		# define the second branch of the Inception module consisting
		# of 1 x 1 and 3 x 3 convolutions
		second = GoogLeNet.conv_module(x, num3x3Reduce, 1, 1, (1,1), chanDim, reg=reg)
		second = GoogLeNet.conv_module(second, num3x3, 3, 3, (1,1), chanDim, reg=reg)
		# define the third branch of the Inception module consisting
		# of 1 x 1 and 5 x 5 convolutions
		third = GoogLeNet.conv_module(x, num5x5Reduce, 1, 1, (1,1), chanDim, reg=reg)
		third = GoogLeNet.conv_module(third, num5x5, 5, 5, (1,1), chanDim, reg=reg)
		# define the fourth branch of the Inception module consisting
		# of the POOLing projection
		fourth = MaxPooling2D((3,3), strides=(1,1), padding="same")(x)
		fourth = GoogLeNet.conv_module(fourth, num1x1Proj, 1, 1, (1,1), chanDim, reg=reg)

		x = concatenate([first, second, third, fourth], axis = chanDim)

		# return the block
		return x

	@staticmethod
	def build_model(width, height, depth, classes, reg=0.0005):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		# model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# define the model input and first CONV module
		inputs = Input(shape=inputShape)
		# CONV => POOL => (CONV * 2) => POOL
		x = GoogLeNet.conv_module(inputs, 64, 7, 7, (2,2), chanDim, reg=reg)
		# POOL1
		x = MaxPooling2D((3,3),strides=(2,2), padding="same")(x)
		# two CONV module (the original paper contains only 1 CONV)
		x = GoogLeNet.conv_module(x, 192, 3, 3, (1,1), chanDim, reg=reg)
		
		# POOL2		
		x = MaxPooling2D((3,3), strides=(2,2), padding="same")(x)

		# the two inception modules
		x = GoogLeNet.inception_mod(x, 64, 96, 128, 16, 32, 32, chanDim, reg=reg)
		x = GoogLeNet.inception_mod(x, 128, 128, 192, 32, 96, 64, chanDim, reg=reg)

		# POOL3
		x = MaxPooling2D((3,3), strides=(2,2), padding="same")(x)

		# the five inception modules
		x = GoogLeNet.inception_mod(x, 192, 96, 208, 16, 48, 64, chanDim, reg=reg)
		x = GoogLeNet.inception_mod(x, 160, 112, 224, 24, 64, 64, chanDim, reg=reg)
		x = GoogLeNet.inception_mod(x, 128, 128, 256, 24, 64, 64, chanDim, reg=reg)
		x = GoogLeNet.inception_mod(x, 112, 144, 288, 32, 64, 64, chanDim, reg=reg)
		x = GoogLeNet.inception_mod(x, 256, 160, 320, 32, 128, 128, chanDim, reg=reg)

		# POOL4
		x = MaxPooling2D((3,3), strides=(2,2), padding="same")(x)
		
		# the two inception modules
		x = GoogLeNet.inception_mod(x, 256, 160, 320, 32, 128, 128, chanDim, reg=reg)
		x = GoogLeNet.inception_mod(x, 384, 192, 384, 48, 128, 128, chanDim, reg=reg)



		# AVG POOL => DO
		x = AveragePooling2D((7,7))(x)
		x = Dropout(0.4)(x)
		

		# flatten , softmax
		x = Flatten()(x)
		x = Dense(classes, kernel_regularizer=l2(reg))(x)
		x = Activation("softmax")(x)

		# create the model
		model = Model(inputs, x, name="GoogLeNet")
		# return the constructed network architecture
		return model
print('sucess')

sucess


In [ ]:
# -*- coding: utf-8 -*-
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras import backend as K
class SRNet_model:
    @staticmethod
    def build_model(width, height, depth, classes):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
        model.add(Conv2D(32, (5, 5), padding="same", strides=(2,2),
                         input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))		
        model.add(Conv2D(32, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(32, (5, 5), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(32, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))    
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (1, 1), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
		#model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (1, 1), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
		
		# first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Dropout(0.5))

		# softmax classifier
        model.add(Dense(12))
        model.add(Activation("softmax"))

		# return the constructed network architecture
        return model


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras import backend as K


class ESLR_model:
    @staticmethod
    def build_model(width, height, depth, classes):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1

		# (CONV => RELU => CONV => RELU ) * 2 => POOL
        model.add(Conv2D(32, (5, 5), padding="same", strides=(2,2),
                         input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))		
        model.add(Conv2D(32, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        #model.add(MaxPooling2D(pool_size=(3, 3),strides=(2,2)))

        model.add(Conv2D(32, (5, 5), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(32, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
        model.add(Dropout(0.25))

		# (CONV => RELU => CONV => RELU) * 2  => POOL
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        #model.add(MaxPooling2D(pool_size=(2, 2)))
        
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
        model.add(Dropout(0.25))
        # (CONV => RELU => CONV => RELU) * 2 => POOL
        model.add(Conv2D(128, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (1, 1), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
		#model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (1, 1), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
		
		# first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Dropout(0.5))

		# softmax classifier
        model.add(Dense(12))
        model.add(Activation("softmax"))

		# return the constructed network architecture
        return model

        

In [ ]:
# VGGNet - 16 trained on image size (224, 224)
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

class VGGNet:
	@staticmethod
	def build_model(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1


		# (CONV => RELU => BN) * 2 => POOL => DO
		model.add(Conv2D(64, (3,3), padding="same", strides=(1,1), input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3,3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
		model.add(Dropout(0.25))

		# (CONV => RELU => BN) * 2 => POOL => DO
		model.add(Conv2D(128, (3,3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(128, (3,3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
		model.add(Dropout(0.25))

		# (CONV => RELU => BN) * 3 => POOL => DO
		model.add(Conv2D(256, (3,3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(256, (3,3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(256, (3,3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
		model.add(Dropout(0.25))

		# (CONV => RELU => BN) * 3 => POOL
		model.add(Conv2D(512, (3,3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(512, (3,3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(512, (3,3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
		model.add(Dropout(0.25))

		# (CONV => RELU => BN) * 3 => POOL
		model.add(Conv2D(512, (3,3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(512, (3,3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(512, (3,3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
		model.add(Dropout(0.25))

		# 
		model.add(Flatten())
		model.add(Dense(4096))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Dropout(0.5))

		model.add(Dense(4096))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Dropout(0.5))

		model.add(Dense(12))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [ ]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

class AlexNet:
	@staticmethod
	def build_model(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# layer 1
		model.add(Conv2D(96, (11, 11), padding="same", strides=(4,4), input_shape=inputShape))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

		# layer 2
		model.add(Conv2D(256, (5,5), padding="same"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Activation("relu"))		
		model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

		# layer 3
		model.add(Conv2D(384, (3,3), padding="same"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Activation("relu"))
		# layer 4
		model.add(Conv2D(384, (3,3), padding="same"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Activation("relu"))
		# layer 5
		model.add(Conv2D(256, (3,3), padding="same"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
		#model.add(Dropout(0.25))

		# layer 6
		model.add(Flatten())
		model.add(Dense(4096))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Activation("relu"))
		model.add(Dropout(0.5))

		# layer 7
		model.add(Dense(4096))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Activation("relu"))
		model.add(Dropout(0.5))

		#layer 8
		model.add(Dense(12))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [ ]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

class ESLR_Net1:
	@staticmethod
	def build_model(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# (CONV => RELU => CONV => RELU ) * 2 => POOL
		model.add(Conv2D(32, (5, 5), padding="same", strides=(2,2),
			input_shape=inputShape))
		model.add(Activation("relu"))	
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		#model.add(MaxPooling2D(pool_size=(3, 3)))
		
		model.add(Conv2D(32, (5, 5), padding="same", strides=(2,2)))
		model.add(Activation("relu"))
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
		

		# (CONV => RELU => CONV => RELU) * 2  => POOL
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		#model.add(MaxPooling2D(pool_size=(2, 2)))
		
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
		
		# (CONV => RELU => CONV => RELU) * 2 => POOL
		model.add(Conv2D(128, (3, 3), padding="same", strides=(2,2)))
		model.add(Activation("relu"))
		model.add(Conv2D(128, (1, 1), padding="same"))
		model.add(Activation("relu"))
		#model.add(MaxPooling2D(pool_size=(2, 2)))

		model.add(Conv2D(128, (3, 3), padding="same", strides=(2,2)))
		model.add(Activation("relu"))
		model.add(Conv2D(128, (1, 1), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		
		
		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(1024))
		model.add(Activation("relu"))
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(12))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [ ]:
# import the necessary packages
'''
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
'''
import tensorflow as tf

class ESLR_Net2:
	@staticmethod
	def build_model(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		inputShape = (height, width, depth)
		chanDim = -1

		# define the model input
		inputs = tf.keras.layers.Input(shape=inputShape)

		# (CONV => RELU => CONV => RELU ) * 2 => POOL
		x = tf.keras.layers.Conv2D(32, (5, 5), padding="same", strides=(2,2))(inputs)
		x = tf.keras.layers.Activation("relu")(x)
		x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)		
		x = tf.keras.layers.Conv2D(32, (3, 3), padding="same")(x)
		x = tf.keras.layers.Lambda(lambda t: tf.nn.crelu(x))(x)
		x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
		#model.add(MaxPooling2D(pool_size=(3, 3)))
		
		x = tf.keras.layers.Conv2D(32, (5, 5), padding="same", strides=(2,2))(x)
		x = tf.keras.layers.Lambda(lambda t: tf.nn.crelu(x))(x)
		x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
		x = tf.keras.layers.Conv2D(32, (3, 3), padding="same")(x)
		x = tf.keras.layers.Lambda(lambda t: tf.nn.crelu(x))(x)
		x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
		x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2))(x)
		x = tf.keras.layers.Dropout(0.25)(x)

		# (CONV => RELU => CONV => RELU) * 2  => POOL
		x = tf.keras.layers.Conv2D(64, (3, 3), padding="same")(x)
		x = tf.keras.layers.Lambda(lambda t: tf.nn.crelu(x))(x)
		x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
		x = tf.keras.layers.Conv2D(64, (3, 3), padding="same")(x)
		x = tf.keras.layers.Lambda(lambda t: tf.nn.crelu(x))(x)
		x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
		#model.add(MaxPooling2D(pool_size=(2, 2)))
		
		x = tf.keras.layers.Conv2D(64, (3, 3), padding="same")(x)
		x = tf.keras.layers.Lambda(lambda t: tf.nn.crelu(x))(x)
		x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
		x = tf.keras.layers.Conv2D(64, (3, 3), padding="same")(x)
		x = tf.keras.layers.Lambda(lambda t: tf.nn.crelu(x))(x)
		x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
		x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2))(x)
		x = tf.keras.layers.Dropout(0.25)(x)
		# (CONV => RELU => CONV => RELU) * 2 => POOL
		x = tf.keras.layers.Conv2D(128, (3, 3), padding="same", strides=(2,2))(x)
		x = tf.keras.layers.Lambda(lambda t: tf.nn.crelu(x))(x)
		x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
		x = tf.keras.layers.Conv2D(128, (1, 1), padding="same")(x)
		x = tf.keras.layers.Lambda(lambda t: tf.nn.crelu(x))(x)
		x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
		#model.add(MaxPooling2D(pool_size=(2, 2)))

		x = tf.keras.layers.Conv2D(128, (3, 3), padding="same", strides=(2,2))(x)
		x = tf.keras.layers.Lambda(lambda t: tf.nn.crelu(x))(x)
		x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
		x = tf.keras.layers.Conv2D(128, (1, 1), padding="same")(x)
		x = tf.keras.layers.Lambda(lambda t: tf.nn.crelu(x))(x)
		x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
		x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
		x = tf.keras.layers.Dropout(0.25)(x)
		
		# first (and only) set of FC => RELU layers
		x = tf.keras.layers.Flatten()(x)
		x = tf.keras.layers.Dense(1024)(x)
		x = tf.keras.layers.Lambda(lambda t: tf.nn.crelu(x))(x)
		x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
		x = tf.keras.layers.Dropout(0.5)(x)

		# softmax classifier
		x = tf.keras.layers.Dense(12)(x)
		x = tf.keras.layers.Activation("softmax")(x)

		# create the model
		model = tf.keras.models.Model(inputs, x)

		# return the constructed network architecture
		return model

In [ ]:
# TDGPerNet - 16 trained on image size (224, 224)
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.layers import concatenate
from keras.regularizers import l2
from keras import backend as K

class ESLR_Net3:
	@staticmethod
	def conv_module(x, K, kX, kY, stride, chanDim, padding="same", reg=0.0005):
		#initialize the CONV, BN, and RELU layer names

		# define a CONV => BN => RELU pattern
		x = Conv2D(K, (kX, kY), strides=stride, padding=padding, kernel_regularizer=l2(reg))(x)
		x = Activation("relu")(x)
		x = BatchNormalization(axis=chanDim)(x)
		
		# return the block
		return x

	@staticmethod
	def pool_mod(x, psize, stride, dropout=0.1):
		maxim = MaxPooling2D(pool_size=psize,strides=stride)(x)
		avrg = AveragePooling2D(pool_size=psize, strides=stride)(x)

		# concatenate maxpooling and averagepooling
		x = concatenate([maxim, avrg])
		x = Dropout(dropout)(x)
		# return the block
		return x

	@staticmethod
	def inception_mod(x, num1x1, num3x3, num5x5, chanDim, reg=0.0005):
		# define the first branch of the Inception module consisting
		# of 1 x 1 convolutions
		first = ESLR_Net3.conv_module(x, num1x1, 1, 1, (1,1), chanDim, reg=reg)
		# define the second branch of the Inception module consisting
		# of 1 x 1 and 3 x 3 convolutions
		#second = TDGPerNet.conv_module(x, num3x3Reduce, 1, 1, (1,1), chanDim, reg=reg)
		second = ESLR_Net3.conv_module(x, num3x3, 3, 3, (1,1), chanDim, reg=reg)
		# define the third branch of the Inception module consisting
		# of 1 x 1 and 5 x 5 convolutions
		#third = TDGPerNet.conv_module(x, num5x5Reduce, 1, 1, (1,1), chanDim, reg=reg)
		third = ESLR_Net3.conv_module(x, num5x5, 5, 5, (1,1), chanDim, reg=reg)

		x = concatenate([first, second, third], axis = chanDim)

		# return the block
		return x

	@staticmethod
	def build_model(width, height, depth, classes, reg=0.0005):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		# model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# define the model input and first CONV module
		inputs = Input(shape=inputShape)
		# 1st conv module
		x = ESLR_Net3.conv_module(inputs, 32, 5, 5, (2,2), chanDim, reg=reg)
		x = ESLR_Net3.conv_module(x, 32, 3, 3, (1,1), chanDim, reg=reg)
		
		# 1st pool module
		x = MaxPooling2D((2,2),strides=(2,2), padding="same")(x)
		x = Dropout(0.1)(x)
		#x = TDGPerNet.pool_mod(x, (2,2), (2,2))
		
		# 2nd conv module
		x = ESLR_Net3.conv_module(x, 64, 3, 3, (1,1), chanDim, reg=reg)
		x = ESLR_Net3.conv_module(x, 64, 3, 3, (1,1), chanDim, reg=reg)
		
		# 2nd pool module
		x = MaxPooling2D((2,2),strides=(2,2), padding="same")(x)
		x = Dropout(0.1)(x)
		#x = TDGPerNet.pool_mod(x, (2,2), (2,2))
		
		# 1st three inception modules
		x = ESLR_Net3.inception_mod(x, 64, 32, 16, chanDim, reg=reg)
		x = ESLR_Net3.inception_mod(x, 96, 48, 16, chanDim, reg=reg)
		x = ESLR_Net3.inception_mod(x, 128, 64, 24, chanDim, reg=reg)
		
		# 3rd pool module
		x = ESLR_Net3.pool_mod(x, (2,2), (2,2))

		# 2nd two inception modules		
		x = ESLR_Net3.inception_mod(x, 160, 64, 24, chanDim, reg=reg)
		x = ESLR_Net3.inception_mod(x, 192, 64, 32, chanDim, reg=reg)
		
		# 4th pool module
		x = ESLR_Net3.pool_mod(x, (2,2), (2,2))

		# AVG POOL => DO
		x = AveragePooling2D((7,7))(x)
		x = Dropout(0.4)(x)
		

		# flatten , softmax
		x = Flatten()(x)
		x = Dense(12, kernel_regularizer=l2(reg))(x)
		x = Activation("softmax")(x)

		# create the model
		model = Model(inputs, x, name="ESLR_Net3")
		# return the constructed network architecture
		return model

In [ ]:
# TDGPerNetr - 16 trained on image size (224, 224)
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.layers import concatenate
from keras.regularizers import l2
from keras import backend as K

class ESLR_Net4:
	@staticmethod
	def conv_module(x, K, kX, kY, stride, chanDim, padding="same", reg=0.0005):
		#initialize the CONV, BN, and RELU layer names

		# define a CONV => BN => RELU pattern
		x = Conv2D(K, (kX, kY), strides=stride, padding=padding, kernel_regularizer=l2(reg))(x)
		x = Activation("relu")(x)
		#x = BatchNormalization(axis=chanDim)(x)
		
		# return the block
		return x

	@staticmethod
	def pool_mod(x, psize, stride):
		maxim = MaxPooling2D(pool_size=psize,strides=stride)(x)
		avrg = AveragePooling2D(pool_size=psize, strides=stride)(x)

		# concatenate maxpooling and averagepooling
		x = concatenate([maxim, avrg])
		#x = Dropout(dropout)(x)
		# return the block
		return x

	@staticmethod
	def inception_mod(x, num1x1, num3x3, num5x5, chanDim, reg=0.0005):
		# define the first branch of the Inception module consisting
		# of 1 x 1 convolutions
		first = ESLR_Net4.conv_module(x, num1x1, 1, 1, (1,1), chanDim, reg=reg)
		# define the second branch of the Inception module consisting
		# of 1 x 1 and 3 x 3 convolutions
		#second = TDGPerNetr.conv_module(x, num3x3Reduce, 1, 1, (1,1), chanDim, reg=reg)
		second = ESLR_Net4.conv_module(x, num3x3, 3, 3, (1,1), chanDim, reg=reg)
		# define the third branch of the Inception module consisting
		# of 1 x 1 and 5 x 5 convolutions
		#third = TDGPerNetr.conv_module(x, num5x5Reduce, 1, 1, (1,1), chanDim, reg=reg)
		third = ESLR_Net4.conv_module(x, num5x5, 5, 5, (1,1), chanDim, reg=reg)

		x = concatenate([first, second, third], axis = chanDim)

		# return the block
		return x

	@staticmethod
	def build_model(width, height, depth, classes, reg=0.0005):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		# model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# define the model input and first CONV module
		inputs = Input(shape=inputShape)
		# 1st conv module
		x = ESLR_Net4.conv_module(inputs, 32, 5, 5, (2,2), chanDim, reg=reg)
		x = ESLR_Net4.conv_module(x, 32, 3, 3, (1,1), chanDim, reg=reg)
		
		# 1st pool module
		x = ESLR_Net4.pool_mod(x, (2,2), (2,2))
		
		# 2nd conv module
		x = ESLR_Net4.conv_module(x, 64, 3, 3, (1,1), chanDim, reg=reg)
		x = ESLR_Net4.conv_module(x, 64, 3, 3, (1,1), chanDim, reg=reg)
		
		# 2nd pool module
		x = ESLR_Net4.pool_mod(x, (2,2), (2,2))
		
		# 1st three inception modules
		x = ESLR_Net4.inception_mod(x, 64, 32, 16, chanDim, reg=reg)
		x = ESLR_Net4.inception_mod(x, 96, 48, 16, chanDim, reg=reg)
		x = ESLR_Net4.inception_mod(x, 128, 64, 24, chanDim, reg=reg)
		
		# 4th pool module
		x = ESLR_Net4.pool_mod(x, (2,2), (2,2))

		# 2nd two inception modules		
		x = ESLR_Net4.inception_mod(x, 160, 64, 24, chanDim, reg=reg)
		x = ESLR_Net4.inception_mod(x, 192, 64, 32, chanDim, reg=reg)
		
		# 5th pool module
		x = ESLR_Net4.pool_mod(x, (2,2), (2,2))

		# AVG POOL => DO
		x = AveragePooling2D((7,7))(x)
		x = Dropout(0.4)(x)
		

		# flatten , softmax
		x = Flatten()(x)
		x = Dense(30, kernel_regularizer=l2(reg))(x)
		x = Activation("softmax")(x)

		# create the model
		model = Model(inputs, x, name="ESLR_Net4")
		# return the constructed network architecture
		return model

In [ ]:
num_features = 32
num_labels = 30
class Model1:
  def build_model(input_shape=(128,128,3)):
    model = Sequential()
    model.add(Conv2D(1, kernel_size=(1, 1), activation='relu', input_shape=input_shape, data_format='channels_last', kernel_regularizer=l2(0.01)))
    model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())

    model.add(Dense(2*2*2*num_features, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(2*2*num_features, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(2*num_features, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_labels, activation='softmax'))
    return model

In [ ]:
gen = ImageDataGenerator(horizontal_flip=True,
                        vertical_flip=True)
train_generator_1 = gen.flow(trainX, trainY, batch_size=batch_size)
test_generator_1 = gen.flow(testX, testY, batch_size=batch_size)

print("[INFO] training network...")
opt = Adam(lr=1e-3, decay=1e-3 / 50)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
H = model.fit_generator(train_generator_1, steps_per_epoch=batch_size, epochs=no_epoch,
                 validation_data=(testX,testY))

In [ ]:
class Model2:
  def build_model(input_shape=(128,128,3)):
    model = Sequential()
    model.add(Conv2D(1, kernel_size=(1, 1), activation='relu', input_shape=input_shape, data_format='channels_last', kernel_regularizer=l2(0.01)))
    model.add(Conv2D(10, (5,5), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Dropout(0.1))
    model.add(Conv2D(10, (5,5), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Dropout(0.15))
    model.add(Conv2D(10, (3,3), activation='relu'))
    model.add(MaxPooling2D()) 
    model.add(Dropout(0.1))

    model.add(Flatten())

    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(30, activation='softmax'))
    return model

  # TODO - replace 

In [ ]:
class Model3:
  def build_model(input_shape=(128,128,3)):
    model = Sequential()
    model.add(Conv2D(1, kernel_size=(1, 1), activation='relu', input_shape=input_shape, data_format='channels_last', kernel_regularizer=l2(0.01)))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())

    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.1))

    model.add(Dense(30, activation='softmax'))
    return model

  # TODO - replace MaxPooling with stochastic pooling

In [ ]:
from keras.applications import VGG16
from keras.models import Model
class Model4:
  def build_model():
    vgg = VGG16(include_top=False, input_shape=(128,128,3))
    k = Flatten()(vgg.output)
    k = Dense(256, activation='relu')(k)
    k = Dropout(0.2)(k)
    k = Dense(128, activation='relu')(k)
    k = Dropout(0.2)(k)
    k = Dense(64, activation='relu')(k)

    pred = Dense(30, activation='softmax')(k)
    model = Model(input=vgg.input, output=pred)
    return model

In [ ]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adadelta
from keras.utils import np_utils
from keras.regularizers import l2 #, activity_l2
class Model5:
  def build_model():
      model = Sequential()
      #input_shape=(66,66, 3) = ukuran gambar
      model.add(Conv2D(8, (3, 3), padding="same", input_shape=(128,128, 3)))
      model.add(Activation("relu"))
      model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
      model.add(Conv2D(16, (3, 3), padding="same"))
      model.add(Activation("relu"))
      model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
      model.add(Conv2D(32, (3, 3), padding="same"))
      model.add(Activation("relu"))
      model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
      #flatten = di jadikan 1 dimensi
      model.add(Flatten())
      #Dense = banyak kelas
      model.add(Dense(30))
      model.add(Activation("softmax"))
      return model

In [ ]:
import numpy as np
import cv2

class gabfilter:
	@staticmethod
	def build_filters():
		filters = []
		ksize = 31
		for theta in np.arange(0, np.pi, np.pi / 16):
			for freq in np.arange(0.1, 1, 0.1):
				#cv2.getGaborKernel(ksize, sigma, theta, freq, gamma, psi, ktype)
				kern = cv2.getGaborKernel((ksize, ksize), 4.0, theta, freq, 0.5, 0, ktype=cv2.CV_32F)
				kern /= 1.5*kern.sum()
				filters.append(kern)
		return filters
	
	def process(img, filters):
		accum = np.zeros_like(img)
		for kern in filters:
			fimg = cv2.filter2D(img, cv2.CV_8UC3, kern)
			np.maximum(accum, fimg, accum)
			return accum

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jun 20 16:47:49 2020

@author: wal
"""

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras import backend as K


class ESLR_Net_Hand_Gesture:
    @staticmethod
    def build_model(width, height, depth, classes):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1

        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
        model.add(Conv2D(32, (7, 7), padding="same", strides=(2,2),
                         input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))		
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Conv2D(32, (7, 7), padding="same", strides=(2,2),
                         input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))		
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Dropout(0.5))




        model.add(Conv2D(32, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(32, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(Conv2D(32, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Conv2D(64, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Conv2D(64, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Dropout(0.5))



        model.add(Conv2D(64, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        #model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Dropout(0.5))

        
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(Conv2D(128, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(Conv2D(128, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(Conv2D(128, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(Conv2D(128, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Dropout(0.5))



        model.add(Flatten())
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Dropout(0.5))

		# softmax classifier
        model.add(Dense(30))
        model.add(Activation("softmax"))

		# return the constructed network architecture
        return model

        


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras import backend as K


class ESLR_Net_Facial_Expression:
    @staticmethod
    def build_model(width, height, depth, classes):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1

        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
            
            
        model.add(Conv2D(32, (3, 3), padding="same", strides=(2,2),
                         input_shape=(128,128,3)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))		
        #model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Dropout(0.5))
      
        
        model.add(Conv2D(32, (1, 1), padding="same", strides=(2,2),
                         input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))		
        model.add(Conv2D(32, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(Conv2D(32, (1, 1), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))		
        model.add(Conv2D(32, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Dropout(0.5))


        model.add(Conv2D(64, (1, 1), padding="same", strides=(2,2),
                         input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(Conv2D(64, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))        
        model.add(Conv2D(64, (1, 1), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(Conv2D(64, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Conv2D(64, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))  
        model.add(Conv2D(64, (3, 3), padding="same", strides=(2,2)))      
        model.add(Activation("relu"))
        model.add(Conv2D(128, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))        
        model.add(Conv2D(128, (1, 1), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(Conv2D(128, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Conv2D(128, (3, 3), padding="same", strides=(2,2)))
        model.add(Activation("relu"))   
        model.add(BatchNormalization(axis=chanDim))
        model.add(Dropout(0.5))

        
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        model.add(Flatten())
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Dropout(0.5))

		# softmax classifier
        model.add(Dense(5))
        model.add(Activation("softmax"))

		# return the constructed network architecture
        return model


In [ ]:
class Segmentation:    
    def thresholdIntegral1(inputMat,s,T = 0.15):
        outputMat=np.zeros(inputMat.shape)
        nRows = inputMat.shape[0]
        nCols = inputMat.shape[1]
        S = int(max(nRows, nCols) / 8)
        s2 = int(S / 4)
        for i in range(nRows):
            y1 = i - s2
            y2 = i + s2
            if (y1 < 0) :
                y1 = 0
            if (y2 >= nRows):
                y2 = nRows - 1
            for j in range(nCols):
                x1 = j - s2
                x2 = j + s2
                if (x1 < 0) :
                    x1 = 0
                if (x2 >= nCols):
                    x2 = nCols - 1
                count = (x2 - x1)*(y2 - y1)
                sum=s[y2][x2]-s[y2][x1]-s[y1][x2]+s[y1][x1]
                if ((int)(inputMat[i][j] * count) < (int)(sum*(1.0 - T))):
                    outputMat[i][j] = 255
        return outputMat
    def segment_steem(image):  
        ratio=1
        #image = cv2.imdecode(np.fromfile('data/im (6).png', dtype=np.uint8), 0)
        image = cv2.resize(image, (int(image.shape[1] / ratio), int(image.shape[0] / ratio)), cv2.INTER_NEAREST)    
        retval, otsu = cv2.threshold(image, 0, 255, cv2.THRESH_OTSU)
        roii = cv2.integral(image)
        for j in range(1):
            thresh = Segmentation.thresholdIntegral1(image, roii)
        print('success')
        return thresh


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from mtcnn.mtcnn import MTCNN
import cv2
y1=0
x1=0
width=0
height=0
x2=0
y2=0
class Segment_face():
    def face_Segment(img):
        detector = MTCNN()
        # detect faces in the image
        faces = detector.detect_faces(img)
        #print(len(faces))
        for i in range(len(faces)):
            # get coordinates
            x1, y1, width, height = faces[i]['box']
            x2, y2 = x1 + width, y1 + height
        img=img[y1:y2, x1:x2]
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        print('success')
        return img


In [ ]:
import cv2
from cv2 import destroyAllWindows
from cv2 import CascadeClassifier
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
class Segment_face():
    def face_Segment(img):

      detector = CascadeClassifier('/content/drive/My Drive/Dataset/haarcascade_frontalface_default.xml')
      # perform face detection
      faces = detector.detectMultiScale(img)
      for i in range(len(faces)):
        # get coordinates
        x1, y1, width, height = faces[i]
        x2, y2 = x1 + width, y1 + height
      data=img[y1:y2, x1:x2]
      gray = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
      print('success')
      return img

In [ ]:
import cv2
import numpy as np
class Segment_skin():
    def skin_Segment(img):

        img_YCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
        YCrCb_mask = cv2.inRange(img_YCrCb, (0, 135, 85), (255,180,135)) 
        YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
    
        YCrCb_result = cv2.bitwise_not(YCrCb_mask)
    
        img2=img
        for i in range(img2.shape[0]):
            for j in range(img2.shape[1]):
                if YCrCb_result[i,j] != 0:
                    img2[i,j,0] = 0
                    img2[i,j,1] = 0
                    img2[i,j,2] = 0
    
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
        YCrCb_mask = cv2.erode(YCrCb_mask, kernel, iterations = 2)
        YCrCb_mask = cv2.dilate(YCrCb_mask, kernel, iterations = 2)
        print('sucess')
        return img2
        #return YCrCb_result
print('success')
